# Iterating over recursive structures with pruning
This notebook illustrates how to iterate over recursive structures while pruning certain structure branches.

## Recursive data structures
We consider the case where we want to iterate over the branches of a tree structure. First, lets define the Tree class. Each node of the tree contains a value which will be used to prune certain branches. When iterating on a tree, we iterate over the children as indicated by the `__recur__` implementation.

In [ ]:
from recur.abc import Recursive


class Node(Recursive):
    
    def __init__(self, value):
        super().__init__()
        self._children = []
        self.value = value
        
    def __recur__(self):
        return self._children
    
    def __repr__(self):
        return str(self.value)

    def add(self, node):
        self._children.append(node)

Now that we have a tree definition, lets build a tree with many branches.

In [ ]:
from random import randint


def add_leafs(node, depth, max_children):
    """Recursively add nodes up to depth levels"""
    
    if depth > 0:
        
        for i in range(1, randint(2, max_children + 1)):
            new_node = node.__class__(i + node.value)
            node.add(new_node)
            add_leafs(new_node, depth - 1, max_children)
         
            
root = Node(0)
add_leafs(root, 4, 3)  # Create the tree with a depth of 4.

# Print all the nodes in preorder.
print([node for node in root])


Now that we have a tree, suppose we wish to obtain all the nodes with a value greater than 4. To do this, we may simply iterate over all nodes and discard the ones we do not want. However, this is inefficient because the value of a node is greater to the value of its parent by construction. Therefore, once we find a node with a values greater than 4, we can safely ignore all its children. For large data structures, this may provide an enormous speed gain.

This may be achieved by specifing a pruning function to the `RecursiveIterator`. A pruning function receives a node (or more generally, a `Recursive` instance) and returns a boolean value. For a given node, if the pruning function returns `False`, the node and all of its children decendants are ignored.

In [ ]:
from time import time

from recur.abc import Order, RecursiveIterator


def prune(node):
    return node.value > 4


# Eliminate nodes by filtering.
print('Filtering: {}'.format([node for node in root if node.value <= 4]))

# Eliminate nodes by pruning gives the save result.
iterator = RecursiveIterator(root, Order.PRE, prune=prune)
print('Pruning: {}'.format([node for node in iterator]))

# For large structure, pruning is more efficient.
large_root = Node(0)
add_leafs(large_root, 10, 3)

start = time()
filtered_nodes = [node for node in large_root if node.value <= 4]
print('Elimination by filtering in {:0.2f} milliseconds.'
      .format(1000 * (time() - start)))

start = time()
iterator = RecursiveIterator(large_root, Order.PRE, prune=prune)
pruned_nodes = [node for node in iterator]
print('Elimination by pruning in {:0.2f} milliseconds.'
      .format(1000 * (time() - start)))

Because the Iterator syntax is a bit heavy, pruning is also supported by the `preoder` and `postorder` convenience functions.

In [ ]:
from recur.abc import postorder, preorder

pruned_nodes = [node for node in preorder(root, prune=prune)]
print('Pruning in preorder: {}'.format(pruned_nodes))

pruned_nodes = [node for node in postorder(root, prune=prune)]
print('Pruning in postorder: {}'.format(pruned_nodes))

## MultiRecursive data structures
Pruning data structures as illustrate above also works for instances of MultiRecursive data structures. Here is an example with a tree where children can iterate over their ancestors. First, lets define the `MultiNode` class which represents a single node in the tree.

In [ ]:
from recur.abc import MultiRecursive


class MultiNode(MultiRecursive):
    
    def __init__(self, value):
        super().__init__()
        self._children = []
        self._parents = []
        self.value = value
        
    def __multirecur__(self, index):
        if index == 0:
            return self._children
        else:
            return self._parents
    
    def __repr__(self):
        return str(self.value)

    def add(self, node):
        self._children.append(node)
        node._parents.append(self)


We can then create a tree and traverse it in preorder and postorder from the root to the leafs or from the leafs to the root. While we could use the `MultiRecursiveIterator` for this, the convenience functions `postorder`, `preorder`, `ancestors` and `decendants` are mush nicer.

In [ ]:
from recur.abc import ancestors, descendants


# Create a tree. We can use the same function because both Node and 
# MultiNode implement the add method.
multi_root = MultiNode(0)
add_leafs(multi_root, 4, 3)

# Print all children and parents.
nodes = [node for node in multi_root]
print('Children of the root: {}'.format(nodes))
parents = [node for node in ancestors(nodes[-1])]
print('Ancestors of a leaf: {}'.format(parents))

# Print pruned children.
pruned_children = [n for n in preorder(descendants(multi_root), prune=prune)]
print('Pruned children of the root: {}'.format(pruned_children))

# Printing the parents of a pruned node yields nothing.
pruned_parents = [n for n in preorder(ancestors(nodes[-1]), prune=prune)]
print('Ancestors of a pruned leaf: {}'.format(pruned_parents))

# Printing the parents of a leaf.
iterator = postorder(ancestors(nodes[-1]), prune=lambda n: n.value < 3)
pruned_parents = [n for n in iterator]
print('Pruned ancestors of a leaf in postorder: {}'.format(pruned_parents))